In [ ]:
import geopandas as gpd
import polars as pl
import polars_st as st
import pyogrio

In [ ]:
# ASGA_FILE_PATH = "data/raw/ASGA/{year}/{region}/"
SA1_FILE_PATH = "/home/andre/git/private/electoralyze/data/raw/ASGA/2021/SA1/SA1_2021_AUST_GDA2020.shp"
VIC_ELECTION = "/home/andre/git/private/electoralyze/data/raw/AEC_boundary/federal/2024/VIC/E_VIC24_region.shp"

In [ ]:
sa1: gpd.GeoDataFrame = pyogrio.read_dataframe(SA1_FILE_PATH)
sa1.sample(3)

sa1_gpd = sa1[["SA1_CODE21", "geometry"]].dropna().assign(geometry=lambda df: df["geometry"].to_crs(4326))

## GPD -> st

In [ ]:
def to_geopolars(gdp_gdf: gpd.GeoDataFrame) -> st.GeoDataFrame:
    st_gdf = st.from_geopandas(gdp_gdf.to_crs(4326)).with_columns(st.geom("geometry").st.set_srid(4326))
    return st_gdf


sa1_st: st.GeoDataFrame = to_geopolars(sa1[["SA1_CODE21", "geometry"]].dropna())
sa1_st

## Comparing simplification

In [ ]:
sa1_gpd_simp = sa1_gpd.assign(geometry=lambda df: df["geometry"].simplify(0.0005))

In [ ]:
sa1_st_simp = sa1_st.with_columns(st.geom("geometry").st.simplify(0.0005))

In [ ]:
# sa1_gpd_simp.head(600).explore()

In [ ]:
def to_geopandas(st_gdf: st.GeoDataFrame) -> gpd.GeoDataFrame:
    gpd_gdf = st_gdf.st.to_geopandas().set_crs(4326)
    return gpd_gdf


sa1_gdf_ = sa1_st_simp.head(600).pipe(to_geopandas)
# sa1_gdf_.explore()
sa1_gdf_

### Comparing centroids

In [ ]:
%%timeit
sa1_gpd.assign(geometry=lambda df: df["geometry"].centroid)

In [ ]:
%%timeit
sa1_st.with_columns(st.geom("geometry").st.centroid())

## Writing

In [ ]:
sa1_file = "temp/sa1_processed.shp"
sa1_gdf: gpd.GeoDataFrame = sa1_st.pipe(to_geopandas)
sa1_gdf.to_file(sa1_file)

In [ ]:
pyogrio.read_dataframe(sa1_file).pipe(to_geopolars)